# Regresión lineal múltiple

Fuente de set de datos: https://www.kaggle.com/shivam2503/diamonds

Revise, antes de comenzar, la descripción del conjunto de datos con el cual va a trabajar.

## Carga de datos

In [ ]:
import pandas as pd
data_frame = pd.read_csv("3.1.6 diamonds.csv")
data_frame.head()

## Exploración inicial

+ Conocer la cantidad de observaciones
+ Tipos de datos
+ Obtención de algunas medidas estadísticas
+ Revisar distribución de los datos
+ Revisar valores nulos
+ Conocer la correlación de variables

In [ ]:
# Cantidad de obsevaciones
data_frame.shape

In [ ]:
data_frame.dtypes

In [ ]:
# Se descarta la primera columna porque no tiene valor predictivo
data_frame = data_frame.drop(["Unnamed: 0"],axis=1)
data_frame.head()

In [ ]:
# Medidas estadísticas
data_frame.describe()

In [ ]:
# Considerando solo las columnas de tipo object
import numpy as np
data_frame.describe(include=[object])

In [ ]:
# Valores nulos
for feature in data_frame.columns:
    print('Total de valores nulos de', feature, '=', data_frame[feature].isna().sum())

In [ ]:
## Correlación de las variables
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,15))
p=sns.heatmap(data_frame.corr(), annot=True,cmap='RdYlGn',square=True)

### Comentarios de gráfico
- La columna 'carat' posee la mayor correlación (0.92)
- x, y, z tienen una correlación bastante alta con el precio

## Análisis de 4 características (carat,x,y,z)

In [ ]:
for col in ['x', 'y', 'z', 'carat']:
    fig, ax =plt.subplots(1, 2, figsize=(20, 6))
    fig.suptitle(col, fontsize=18)
    sns.distplot(data_frame[col], ax=ax[0], kde=False)
    data_frame[[col]+['price']].plot.scatter(x=col, y='price', ax=ax[1])
    plt.show()

### Comentarios

De las gráficas anteriores se visualiza que hay obsevaciones que tienen valores 0 en estas características, lo que no tiene mucho sentido porque representan el ancho, alto y profundidad del diamante.

Dada la conclusión anterior, se eliminarán estas observaciones

In [ ]:
data_frame = data_frame[(data_frame[['x','y','z']] != 0).all(axis=1)]
data_frame.shape

Una vez que se han eliminado las observaciones quedan 53.920 observaciones

Ahora se vuelven a ver las distribuciones.

In [ ]:
# Visualizamos la distribución de las variables
columnas = ['x', 'y', 'z', 'carat', 'depth']
for col in columnas:
    fig, ax =plt.subplots(1, 2, figsize=(20, 6))
    fig.suptitle(col, fontsize=18)
    sns.distplot(data_frame[col], ax=ax[0], kde=False)
    data_frame[[col]+['price']].plot.scatter(x=col, y='price', ax=ax[1])
    plt.show()

## Tratamiento de variables categóricas

Es importante recordar que estas variables no han sido consideradas en el análisis de correlación anterior porque no son numéicas; así es que conveniente considerarlas

Primero se van a revisar los valores únicos de cada variable categórica.

Luego se van a transformar usando onehot-encoder

In [ ]:
data_frame['cut'].unique()

In [ ]:
data_frame['color'].unique()

In [ ]:
data_frame['clarity'].unique()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['cut'])], remainder='passthrough')
#X = np.array(ct.fit_transform(data_frame))
X = ct.fit_transform(data_frame)
pd.DataFrame(X)

## Desafío

Repita la técnica de transformación con las otras 2 columnas y comente los resultados

## Creando, validando y evaluando el modelo

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Se separan las variables predictoras y la predecida

x = data_frame[['x','y', 'z', 'carat']] # Variables independientes
y = data_frame['price'] # Variable dependiente

X_train, X_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.2, random_state=29)
# Tipo de modelo
modelo = LinearRegression()

# Entrenamiento del modelo
modelo.fit(X_train, y_train)

# Validación del modelo
y_pred = modelo.predict(X_test)

# Evaluación del modelo
print("R2 :%.2f" % r2_score(y_test, y_pred))
print("MSE: %.2f" % mean_squared_error(y_pred, y_test))
print("Score del modelo : %.2f" % modelo.score(X_train, y_train))

## Realizando predicciones

In [ ]:
print(modelo.predict([[4.15, 3.95, 2.25, 0.29]]))


## Mejoras

Ya se generó un modelo de predicción del precio de un diamante considerando su alto, ancho, profundidad y peso del diamante.

Sin embargo, estas variables no están normalizadas. El desafío ahora es repetir el ejercicio considerando las variables normalizadas.

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
data_selected = data_frame[['x','y', 'z', 'carat']]
scaled_data = sc.fit_transform(data_selected)
pd.DataFrame(scaled_data).head()

In [ ]:
scaled_data.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaled_data, y, 
                                                    test_size=0.2, random_state=29)
# Tipo de modelo
modelo_x = LinearRegression()

# Entrenamiento del modelo
modelo_x.fit(X_train, y_train)

# Validación del modelo
y_pred = modelo_x.predict(X_test)

# Evaluación del modelo
print("R2 :%.2f" % r2_score(y_test, y_pred))
print("MSE: %.2f" % mean_squared_error( y_test, y_pred))
print("Score del modelo : %.2f" % modelo_x.score(X_train, y_train))

In [ ]:
# Se realiza la misma predicción anterior
print(modelo.predict([[4.15, 3.95, 2.25, 0.29]]))

In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 6)

plt.plot(y_pred[:100], label='Predicted')
plt.plot(y_test.values[:100], label='Actual')
plt.ylabel('Price')

plt.legend()
plt.show()